In [1]:
import spacy
from spacy.language import Language

In [84]:
nlp = spacy.load("en_core_web_trf")

# Accomodating longer sentences
nlp.max_length = 2000000

TypeError: load() got an unexpected keyword argument 'enable'

In [83]:
nlp.pipeline

[('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x7fc211947e80>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x7fc210d59240>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7fc210297220>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7fc2106a0800>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7fc2101e3e40>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7fc210297f40>)]

In [190]:
@Language.component("custom_sentencizer")
def custom_sentencizer(doc: str) -> object:
    """ Custom sentencizer to ignore brackets as sentence boundaries.

    Parameters
    ----------
    doc : str
        Text to process.

    Returns
    -------
    doc : spacy.Doc
        spacy document with sentence boundaries.

    Notes
    -----

    """
    # Exception tokens
    exceptions = ["mr.", "ms.", "mrs.", "adv.", "advs.", "sr.",
                  "dr.", "m.", "crl.", "a.", "advocate.", "advocates.",
                  "w.", "p.", "fir.", "ltd."]

    # The last token cannot start a sentence
    for i, token in enumerate(doc[:-2]):
        
        if token.text[-1] in ".?!:":

            doc[i+1].is_sent_start = True
            
            num_punct = sum([char in ".?!:" for char in token.text])
            
            if  num_punct > 1 and num_punct < len(token.text):
                doc[i+1].is_sent_start = False
            
            if not doc[i+1].text[0].isupper():
                doc[i+1].is_sent_start = False

            if doc[i+1].text[-1] == '.' and len(doc[i+1].text) == 2:
                doc[i+1].is_sent_start = False
                
            if doc[i].text[-1] == '.' and len(doc[i].text) == 2:
                doc[i+1].is_sent_start = False

            if token.text.lower() in exceptions:
                doc[i+1].is_sent_start = False

            if (doc[i].is_sent_start is True):
                doc[i+1].is_sent_start = False
                
    doc[-1].is_sent_start = False

    return doc

In [191]:
nlp_1 = spacy.blank('en')

In [192]:
nlp_1.add_pipe("custom_sentencizer")

<function __main__.custom_sentencizer(doc: str) -> object>

In [193]:
nlp_1.pipeline

[('custom_sentencizer',
  <function __main__.custom_sentencizer(doc: str) -> object>)]

In [197]:
sent = ["This is a test. Will the sentencizer work?", "This is a second sentence where A.K. Rampur went somewhere.           .... This is bad english I.P.C. is a code of conduct okay?"]

In [198]:
sent_nlp = nlp_1.pipe(sent)

In [199]:
for t in nlp_1.pipe(sent):
    for s in t.sents:
        print(s)

This is a test.
Will the sentencizer work?
This is a second sentence where A.K. Rampur went somewhere.           ....
This is bad english I.P.C. is a code of conduct okay?


In [141]:
for sent in sent_nlp:
    print(sent)

In [ ]:
pr